# Ejercicios Pair Programming 
## Lección 14 - Regresión Logística Preprocesado

Usando el mismo dataset que usatéis ayer, los objetivos de los ejercicios de hoy son:

In [154]:
#Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Establecer tamaño gráficas
plt.rcParams["figure.figsize"] = (10,10)

# Warning
import warnings
warnings.filterwarnings('ignore')

In [155]:
# Cargamos el dataframe
df = pd.read_csv("superstore_data.csv", index_col=0)

Estandarizar las variables numéricas de vuestro set de datos

Codificar las variables categóricas. Recordad que tendréis que tener en cuenta si vuestras variables tienen orden o no.

Chequear si vuestros datos están balanceados. En caso de que no lo estén utilizad algunas de las herramientas aprendidas en la lección para balancearlos.

Guardad el dataframe con los cambios que habéis aplicado para utilizarlo en la siguiente lección.

Happy coding 📊📈📉